In [6]:
from helper import *
from funcs import *

**tf.tsv**

always predict A-coref as True and B-coref as False

In [7]:
f_test = open("gap-test.tsv", "r")
result = open("tf.tsv", 'w')
test_data = f_test.readlines()

for line in test_data[1:]:
    attrs = line.split('\t')
    result.write("\t".join([attrs[0]] + ['True', 'False']) + "\n")

f_test.close()
result.close()

In [8]:
!python gap_scorer.py --gold_ts=gap-test.tsv --system_tsv=tf.tsv

gap_scorer.py:126: DeprecationWarning: 'U' mode is deprecated
  with open(filename, 'rU') as f:
Overall recall: 51.8 precision: 45.9 f1: 48.7
		tp 918	fp 1082
		fn 855	tn 1145
Masculine recall: 51.0 precision: 45.3 f1: 48.0
		tp 453	fp 547
		fn 436	tn 564
Feminine recall: 52.6 precision: 46.5 f1: 49.4
		tp 465	fp 535
		fn 419	tn 581
Bias (F/M): 1.03



**closer.tsv**

Predict closer coreference to pronoun as True

In [9]:
f_test = open("gap-test.tsv", "r")
result = open("closer.tsv", 'w')
test_data = f_test.readlines()

for line in test_data[1:]:
    attrs = line.split('\t')
    tf = ['True', 'False']
    if abs(int(attrs[3]) - int(attrs[5])) > abs(int(attrs[3]) - int(attrs[8])):
        tf = ['False', 'True']

    result.write("\t".join([attrs[0]] + tf) + "\n")

f_test.close()
result.close()

In [10]:
!python gap_scorer.py --gold_ts=gap-test.tsv --system_tsv=closer.tsv

gap_scorer.py:126: DeprecationWarning: 'U' mode is deprecated
  with open(filename, 'rU') as f:
Overall recall: 50.1 precision: 44.4 f1: 47.1
		tp 888	fp 1112
		fn 885	tn 1115
Masculine recall: 51.6 precision: 45.9 f1: 48.6
		tp 459	fp 541
		fn 430	tn 570
Feminine recall: 48.5 precision: 42.9 f1: 45.5
		tp 429	fp 571
		fn 455	tn 545
Bias (F/M): 0.94



In [11]:
print_sents(random=True)

development-1937 Edward E. Miller, U.S. Representative from Illinois Frosty Peters, American football player Frank Phillips, founder of the Phillips Petroleum Company Amy Higgins, screenwriter John JR Robinson (born 1954), one of the most recorded drummers in history Phyllis Thede (born 1954), lived here as a child; since 2009 she has served as an Iowa state representative Marcia Wallace (1942--2013), actress

development-1833 `` Alexander's writing and photo features have been recognized with several awards from the Society of Professional Journalists. Her portraits of Alice Walker and Gloria Steinem were featured on the cover of the Fall 2009 issue of Ms. magazine, commemorating Steinem's 75th birthday. A second portrait of Walker by Alexander was the cover photograph of the October 2010 Writer's Digest, and discussed by Walker on her personal blog.

development-1765 During the restaurant's 1980s heyday at the top of the Zagat's survey, it was mentioned in Wall Street (1987) by Gordo

In [12]:
import pandas as pd

In [13]:
train_data = pd.read_csv('gap-development.tsv', sep='\t')
test_data = pd.read_csv('gap-test.tsv', sep='\t')
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   ID              2000 non-null   object
 1   Text            2000 non-null   object
 2   Pronoun         2000 non-null   object
 3   Pronoun-offset  2000 non-null   int64 
 4   A               2000 non-null   object
 5   A-offset        2000 non-null   int64 
 6   A-coref         2000 non-null   bool  
 7   B               2000 non-null   object
 8   B-offset        2000 non-null   int64 
 9   B-coref         2000 non-null   bool  
 10  URL             2000 non-null   object
dtypes: bool(2), int64(3), object(6)
memory usage: 144.7+ KB


**subject.tsv**

Predict farther coreference pronoun as True when pronoun is subject pronoun (he, she).
Predict closer coreference pronoun as True when pronoun is not subject pronoun, which is object or possessive pronoun(her, his, him).

In [14]:
pronouns = set(train_data['Pronoun'])
pronouns

{'He', 'Her', 'His', 'She', 'he', 'her', 'him', 'his', 'she'}

In [15]:
result = open("subject.tsv", 'w')

for attrs in test_data.iloc:
    pro, pro_id = attrs[2:4]
    A, A_id = attrs[4:6]
    B, B_id = attrs[7:9]

    if pro.lower() in ['he', 'she']:
        if abs(pro_id - A_id) > abs(pro_id - B_id):
            tf = ['TRUE', 'FALSE']
        else:
            tf = ['FALSE', 'TRUE']
    else:
        if abs(pro_id - A_id) > abs(pro_id - B_id):
            tf = ['FALSE', 'TRUE']
        else:
            tf = ['TRUE', 'FALSE']

    result.write("\t".join([attrs[0]] + tf) + "\n")

f_test.close()
result.close()

In [16]:
!python gap_scorer.py --gold_ts=gap-test.tsv --system_tsv=subject.tsv

gap_scorer.py:126: DeprecationWarning: 'U' mode is deprecated
  with open(filename, 'rU') as f:
Overall recall: 58.3 precision: 51.6 f1: 54.8
		tp 1033	fp 967
		fn 740	tn 1260
Masculine recall: 59.4 precision: 52.8 f1: 55.9
		tp 528	fp 472
		fn 361	tn 639
Feminine recall: 57.1 precision: 50.5 f1: 53.6
		tp 505	fp 495
		fn 379	tn 621
Bias (F/M): 0.96



In [17]:
false_case = 0
for attrs in test_data.iloc:
    if attrs[6] == attrs[9]:
        false_case += 1

false_case/len(test_data)

0.1135

# subject.tsv

draw a relation between two proper nouns with sentence structure

(e.g. Which one is the subject?)

In [18]:
import nltk
sent = "In the next pair, Mikhaylov equalled Grishin's time and they shared Olympic gold."
tagged_sent = nltk.pos_tag(nltk.word_tokenize(sent))

Classify train_data into several categories

1. When words are inside parenthesis
2. When words appear after the pronoun
 * When the pronoun is subject pronoun
 * When the pronoun is object/possessive pronoun
3. When the both coreferences are false
4. When only one word is subject
5. When the both words are subject
6. When none of the words are subject
    

In [19]:
import pandas as pd 

train_data = pd.read_csv('gap-development.tsv', sep='\t')

## 1. When words are inside parenthesis
Assume when word is inside parenthesis, coreference is always false

In [20]:
parenthesis_data = []

def isparenthesis(row):
    sent = row['Text']
    A = row['A']
    A_id = row['A-offset']
    B = row['B']
    B_id = row['B-offset']

    return (not (sent[A_id-1] == "(" and sent[A_id+len(A)] == ")"),
            not (sent[B_id-1] == "(" and sent[B_id+len(B)] == ")"))

def only_parenthesis(row):
    return sum(isparenthesis(row)) == 1

In [21]:
parenthesis_data = pd.DataFrame(parenthesis_data, columns=train_data.columns)

In [22]:
print_sents(datas={'manual':True, 'data':parenthesis_data}, num_lines = 10)

result : 22 out of 31 sentences are predicted correct --> accuracy : 71%

## 2. When words appear after the pronoun
Assume coreference of words coming after the pronoun is false

In [23]:
after_pronoun = []

for row in train_data.iloc:
    if row['Pronoun-offset'] < row['A-offset'] or row['Pronoun-offset'] < row['B-offset']:
        after_pronoun.append(row)

after_pronoun = pd.DataFrame(after_pronoun, columns=train_data.columns)

In [24]:
after_subject_pronoun = after_pronoun[after_pronoun['Pronoun'].isin(['he', 'she', 'He', 'She'])]
after_other_pronoun = after_pronoun[after_pronoun['Pronoun'].isin(['his', 'her', 'him', 'His', 'Her', 'Him'])]

In [25]:
print_sents(datas={'manual':True, 'data':after_subject_pronoun}, num_lines = 10)

development-1581 Jenkins then joined the Reform Party, but Endean was selected as the party's candidate instead. Endean represented the Auckland electorates of Parnell from 1930 to 1938, and then Remuera from 1938 to 1943. When the Reform Party merged with United to form the National Party in 1936, Endean became a National Party member. He was controversial within the National Party and in the party's 50-year history written by Barry Gustafson, it is remarked that in 1938, there was ``some resistance to the National MP'' in the Remuera electorate.

development-501 In the prequel comic to Injustice: Gods Among Us, Helena is a member of Batman's insurgency , that combats Superman's One-Earth regime, she was very close to Kate Kane and Renee Montoya , Helena is killed in a battle against Wonder Woman , thus angering Kate and strengthening her resolve to fight the regime.

development-1840 Livsey eventually cracks under the pressure and insists that she will reveal the truth to anyone othe

In [26]:
def after_pronoun_predict(data):
    A_pred = []
    B_pred = []
    for row in data.iloc:
        A_value = False if row['A-offset'] > row['Pronoun-offset'] else True
        B_value = False if row['B-offset'] > row['Pronoun-offset'] else True

        A_pred.append(A_value)
        B_pred.append(B_value)

    data['A-pred'] = A_pred
    data['B-pred'] = B_pred

In [27]:
after_pronoun_predict(after_subject_pronoun)
after_pronoun_predict(after_other_pronoun)

<ipython-input-26-ceb8dec2fe17>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['A-pred'] = A_pred
<ipython-input-26-ceb8dec2fe17>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['B-pred'] = B_pred


In [28]:
print(accuracy(after_subject_pronoun))
print(accuracy(after_other_pronoun))

0.411214953271028
0.2133676092544987


Result : It doesn't really matter whether a word comes after the pronoun or not

## 3. When the both word coreference is false

Let's find out what they have in common.

In [29]:
false_cases = 0
for row in train_data.iloc:
    if row['A-coref'] == row['B-coref']:
        false_cases += 1

print(false_cases/len(train_data))
print(false_cases)

0.1005
201


There are more than 10 percent of these kind of cases

In [30]:
falses_data = train_data[(train_data['A-coref'] == False) & (train_data['B-coref'] == False)]

In [31]:
print_sents(datas={'manual':True, 'data':falses_data}, num_lines = 10)

development-1006 Robert Lowth pronounced it 'the best edition of Horace ever yet delivered to the world. ' In 1788 Zeunius incorporated in an edition of Horace all Baxter's and Gesner's notes. Baxter's Horace includes abuse of Richard Bentley. In 1719 he published his dictionary of British antiquities under the title of Glossarium Antiquitatum Britannicarum, sive Syllabus Etymologicus Antiquitatum Veteris Britanniae atque Iberniae temporibus Romanorum.

development-924 Many middle- and back-markers also made significant gains in the first lap, especially Verstappen, who overtook nine cars in the space of two minutes earning the title of ``Driver of the Day''. The early action was however soon neutralised following contact between P*rez and Stroll which left Stroll spun out and beached in a gravel trap with a ruined set of tyres. The removal of his car warranted a virtual safety car during which Vettel pitted for slicks.

development-1104 William Fane de Salis was an uncle, Lord Bishop 

We can see that there are some patterns in both false sentences

1. There is another pronoun appeared before.
2. when both word is behind the pronoun, and the subject of the sentence is NNP.

1. There is another pronoun appeared before

In [32]:
def prior_pronoun(row):
    token_dict = tokenized_index(row)

    words = token_dict['words']
    word_index = token_dict['word_index']

    male = ['He', 'His', 'Him', 'he', 'his', 'him']
    female = ['She', 'Her', 'she', 'her']

    pro_id = word_index[row['Pronoun-offset']]

    if row['Pronoun'] in male:
        for word in words[:pro_id]:
            if word in male:
                return True
        return False
    elif row['Pronoun'] in female:
        for word in words[:pro_id]:
            if word in female:
                return True
        return False

    else:
        print("Unexpected pronoun : {}", row['Pronoun'])

In [33]:
prior_pronoun_data = apply_model(train_data, [prior_pronoun], lambda x: (False, False))

1141 0.10429447852760736


In [34]:
def another_subject(row):
    token_dict = tokenized_index(row)

    tagged_sents = token_dict['tagged_sents']
    sents = token_dict['sents']
    word_index = token_dict['word_index']
    sent_index = token_dict['sent_index']

    pro_sent_id = sent_index[word_index[row['Pronoun-offset']]]

    def first_noun(sent):
        for word, tag in sent:
            if tag.startswith('NN') or tag.startswith('PR'):
                return (word, tag)

    word, tag = first_noun(tagged_sents[pro_sent_id])
    if tag.startswith('NNP') and not word in [row['A'].split()[0], row['B'].split()[0]]:
        return True

    return False

In [35]:
def both_latter(row):
    if row['A-offset'] > row['Pronoun-offset'] and row['B-offset'] > row['Pronoun-offset']:
        return True

    return False

mask = train_data.apply(both_latter, axis=1)
both_latter_data = train_data[mask]

In [36]:
another_subject_data = apply_model(train_data, [both_latter, another_subject], lambda x: (False, False))

48 0.5833333333333334


In [37]:
print_sents(datas={'manual':True, 'data':another_subject_data}, num_lines = 5)

development-1619 St John's Church has special association with the activity of individuals within the South Sea Islander community and Anglican Church whose activities have been significant within the history of the region and state of Queensland, including Bishop George Halford, consecrated as Anglican Bishop of Rockhampton in 1909, who was greatly respected for his long and active support of the South Sea Islander community; and Frank Solomon, the South Sea Islander community's Vicar's representative on the first St John's Mission Church Committee, who dedicated his life to this church.

development-395 More reliable sources suggest that the ``Santa Cruz'' in actuality refers to the crossroads located in the Meadow of Mudela (so called for its proximity to the pass of Muradal, one of the entrances to the Central Meseta through the Andalusian Plain). On January 30, 1538, Don *lvaro the Elder bought off from Carlos I the towns of Santa Cruz de Mudela and Viso del Puerto (Muradal), unde

## 4. When only one word is subject
Assume that subject word is always true, and other word is always false

In [38]:
subject_data = train_data[train_data['Pronoun'].isin(['he', 'she', 'He', 'She'])]
other_data = train_data[train_data['Pronoun'].isin(['his', 'him', 'her', 'His', 'Him', 'Her'])]
print(len(subject_data))
print(len(other_data))

801
1199


In [39]:
overall_analysis(train_data, [only_subject], [word_subject])

100%|██████████| 9/9 [01:23<00:00,  9.27s/it]


,Pronoun,Length,Accuracy0
0,He,87,0.885057
1,She,115,0.921739
2,His,18,0.777778
3,Her,19,0.736842
4,he,139,0.791367
5,she,147,0.768707
6,his,306,0.767974
7,him,58,0.431034
8,her,270,0.648148
9,total,1159,0.749784


## 5. When both words are subject
Assume that latter word is True, while the other is False

In [40]:
both_subject_data = train_data.apply(both_subject, axis=1)
both_subject_data = train_data[both_subject_data]
len(both_subject_data)

202

In [41]:
print_sents(datas={'manual':True, 'data':both_subject_data}, num_lines = 10)

development-1428 Dudley's son, John H. Dean, also played football for Harvard and was captain of the 1933 team. Dudley became friends with Joseph P. Kennedy as the two men spent summers together in Cohasset, Massachusetts. In 1922, Dudley led an unsuccessful campaign to have Kennedy admitted to the Cohasset Golf Club, and several of his letters lobbying for Kennedy's admission were published in the book, The Fitzgeralds and the Kennedys, by Doris Kearns Goodwin.

development-1934 Whilst Lynette Porter thought that Gwen became a ``fallen woman'' through her affair with Owen, she observed that this made Gwen's role in relation to Jack in ``End of Days'', analogous to that of Mary Magdalene to Jesus, reinforcing a prevalent biblical subtext. Gwen keeps a vigil at Jack's ``Torchwood tomb'' and like Magdalene is rewarded for her loyalty by being the first to lay eyes upon the risen saviour.

development-1090 Just 2 months later at the Winston 500 at Talladega, racing to the checkered flag, 

In [42]:
overall_analysis(train_data, [both_subject], [latter_word, former_word, closer_word, farther_word])

100%|██████████| 9/9 [00:29<00:00,  3.24s/it]


,Pronoun,Length,Accuracy0,Accuracy1,Accuracy2,Accuracy3
0,He,10,0.500000,0.400000,0.500000,0.400000
1,She,14,0.571429,0.428571,0.500000,0.500000
2,His,4,0.750000,0.250000,0.750000,0.250000
3,Her,0,0.000000,0.000000,0.000000,0.000000
4,he,18,0.833333,0.055556,0.833333,0.055556
5,she,25,0.640000,0.200000,0.640000,0.200000
6,his,42,0.690476,0.261905,0.690476,0.261905
7,him,22,0.181818,0.681818,0.181818,0.681818
8,her,67,0.597015,0.358209,0.611940,0.343284
9,total,202,0.594059,0.331683,0.594059,0.331683


## 6. When both words are not subject


In [43]:
none_subject_data = train_data.apply(none_subject, axis=1)
none_subject_data = train_data[none_subject_data]
len(none_subject_data)

639

In [44]:
overall_analysis(train_data, [none_subject], [latter_word, former_word, closer_word, farther_word])

100%|██████████| 9/9 [00:39<00:00,  4.40s/it]


,Pronoun,Length,Accuracy0,Accuracy1,Accuracy2,Accuracy3
0,He,41,0.390244,0.463415,0.439024,0.414634
1,She,50,0.280000,0.420000,0.240000,0.460000
2,His,11,0.181818,0.454545,0.181818,0.454545
3,Her,19,0.368421,0.368421,0.368421,0.368421
4,he,78,0.423077,0.371795,0.423077,0.371795
5,she,77,0.337662,0.402597,0.324675,0.415584
6,his,148,0.452703,0.290541,0.506757,0.236486
7,him,18,0.555556,0.222222,0.555556,0.222222
8,her,197,0.502538,0.319797,0.507614,0.314721
9,total,639,0.428795,0.347418,0.441315,0.334898


In [45]:
overall_analysis(train_data, [none_subject, both_latter, another_subject], [lambda x: (False, False)])

100%|██████████| 9/9 [00:42<00:00,  4.74s/it]


,Pronoun,Length,Accuracy0
0,He,0,0.000000
1,She,0,0.000000
2,His,0,0.000000
3,Her,0,0.000000
4,he,2,0.500000
5,she,2,1.000000
6,his,13,0.692308
7,him,0,0.000000
8,her,4,0.750000
9,total,21,0.714286


In [86]:
print_sents(datas={'manual':True, 'data':none_subject_data[none_subject_data.apply(lambda row: row['Pronoun'] in ['He','She','His','Her'], axis=1)]}, num_lines = 10)

development-156 Homer's epics stand at the beginning of the western canon of literature, exerting enormous influence on the history of fiction and literature in general. Alexander the Great (356BC-323BC) was a Greek king of Macedon and the creator of one of the largest empires in ancient history. He was tutored by the philosopher Aristotle and, as ruler, broke the power of Persia, overthrew the Persian king Darius III and conquered the Persian Empire.

development-291 Llewellyn failed to take a wicket in this first Test and was promptly omitted from the remainder of the series but responded by performing impressively in the 1897--98 and 1898--99 Currie Cups, which led to his recall to the national team for the first Test of the 1898--99 series against England. Llewellyn impressed by taking five wickets but was surprisingly left out of the second Test. At the end of the 1898--99 series Llewellyn, perturbed by the actions of the selectors and seeking financial security, left South Africa

1. when two words are close to each other, referred words are more likely to be coreferred word.

 * {True word} (,?) and {Pronoun}


In [47]:
def CC_pronoun(row):
    token_dict = tokenized_index(row)

    tagged_words = token_dict['tagged_words']
    word_index = token_dict['word_index']

    pro_index = word_index[row['Pronoun-offset']]
    A_index = word_index[row['A-offset']]
    B_index = word_index[row['B-offset']]

    def coref(index):
        if index+5 >= len(tagged_words):
            return False
        return (tagged_words[index+1][1] == 'CC' and pro_index == index + 2) or \
               (tagged_words[index+2][1] == 'CC' and pro_index == index + 3)

    return coref(A_index), coref(B_index)

def exist_CC_pronoun(row):
    return sum(CC_pronoun(row)) == 1

In [48]:
apply_model(train_data, [none_subject, exist_CC_pronoun], CC_pronoun)

15 0.9333333333333333


,ID,Text,Pronoun,Pronoun-offset,A,A-offset,A-coref,B,B-offset,B-coref,URL,A-pred,B-pred
478,development-479,Sarah Wadsworth (17 Mar 1649 - ) m. Jonathan A...,his,260,William,222,False,John Wadsworth,242,True,http://en.wikipedia.org/wiki/William_Wadsworth...,False,True
535,development-536,The orchestra moved on to the Mound Club in St...,His,267,George Burns,237,False,Tal Henry,253,True,http://en.wikipedia.org/wiki/Tal_Henry,False,True
607,development-608,Little Fish is a 2005 Australian film directed...,her,276,Liz Watts,222,False,Cate Blanchett,257,True,http://en.wikipedia.org/wiki/Little_Fish_(film),False,True
633,development-634,The royalties from the performances of the wor...,his,310,Frankie Laine,277,False,Paul Weston,296,True,http://en.wikipedia.org/wiki/Jalousie_'Tango_T...,False,True
694,development-695,They moved to Go**bki near Warsaw with their t...,her,289,Krystyna Izbicki,256,False,Anna Zofia,274,True,http://en.wikipedia.org/wiki/Jerzy_and_Irena_K...,False,True
756,development-757,"Immediately after Blackwood's penalty, wide re...",he,332,Theismann,277,False,Garrett,319,True,http://en.wikipedia.org/wiki/Super_Bowl_XVII,False,True
778,development-779,Not only was it the first scholarly edition of...,Her,290,Mary Lascelles,257,True,Jane Austen,274,False,http://en.wikipedia.org/wiki/Jane_Austen,False,True
1212,development-1213,Truth or Dare by Madonna is a lifestyle brand ...,her,321,Material Girl,258,False,Madonna,309,True,http://en.wikipedia.org/wiki/Truth_or_Dare_by_...,False,True
1320,development-1321,"However, Archie Mitchell uses Suzy to plant DN...",she,317,Queen Victoria,278,False,Suzy,308,True,http://en.wikipedia.org/wiki/Suzy_Branning,False,True
1468,development-1469,"He declared that if James were dead, he would ...",his,313,James,232,False,Brokesby,300,True,http://en.wikipedia.org/wiki/Francis_Brokesby,False,True


2. Located in the beggining of a sentence.

In [75]:
def begin_sentence(row):

    token_dict = tokenized_index(row)
    tagged_sents = token_dict['tagged_sents']

    Subjects = []

    def first_NP(sent, index):

        first_np = ""
        while sent[index][1].startswith('NN') or sent[index][1].startswith('W') or sent[index][1] in ["'s", 'DT', 'IN', 'CD']:
            while sent[index][1].startswith('NNP'):
                first_np += sent[index][0] + " "
                index += 1
            if len(first_np) > 0:
                Subjects.append(first_np)

            if index < len(sent) - 1:
                index += 1
            else:
                break

    for sent in tagged_sents:
        
        index = 0
        first_np = first_NP(sent, index)

    def exist(word):
        for first_word in Subjects:
            if word in first_word:
                return True
            elif first_word in word:
                return True
        return False

    return (exist(row['A']), exist(row['B']))

def begin_sentence_word(row):
    return sum(begin_sentence(row)) == 1

In [154]:
def most_occurance(row):

    token_dict = tokenized_index(row)
    tagged_words = token_dict['tagged_words']

    index=0
    NPs = []
    NP_dict = {}

    while index < len(tagged_words):
        np = ""
        while index < len(tagged_words) and tagged_words[index][1].startswith('NNP'):
            np += tagged_words[index][0] + " "
            index += 1
        
        if len(np) > 0:
            NPs.append(np[:-1])

        index += 1

    def occurance(word):
        occ = 0
        for w in NPs:
            if w in word:
                occ += 1

        return occ

    for word in set(NPs):
        NP_dict[word] = occurance(word)

    max_occ = sorted(NP_dict.items(), key = (lambda x: x[1]), reverse=True)[0]

    if max_occ[1] > 1:
        return [NP for NP in NP_dict if NP_dict[NP] == max_occ[1]]
    else:
        return None

def more_occurance(row):

    most_occs = most_occurance(row)

    if most_occs == None:
        return (True, True)

    def exist(word, words):
        for w in words:
            if word in w or w in word:
                return True
        return False

    return (exist(row['A'], most_occs), exist(row['B'], most_occs))

def only_most_occurance(row):
    return sum(more_occurance(row)) == 1

def none_most_occurance(row):
    return sum(more_occurance(row)) == 0

def both_most_occurance(row):
    return more_occurance(row) == (True, True)

In [141]:
overall_analysis(train_data, [none_subject, only_most_occurance], [more_occurance])

100%|██████████| 9/9 [01:22<00:00,  9.20s/it]


,Pronoun,Length,Accuracy0
0,He,20,0.700000
1,She,16,0.562500
2,His,4,0.750000
3,Her,9,0.555556
4,he,26,0.653846
5,she,34,0.441176
6,his,59,0.576271
7,him,9,0.555556
8,her,92,0.597826
9,total,269,0.583643


In [147]:
none_occurance_data = none_subject_data[none_subject_data.apply(none_most_occurance, axis=1)]

In [155]:
both_occurance_data = none_subject_data[none_subject_data.apply(both_most_occurance, axis=1)]

In [156]:
print(len(none_occurance_data))
print(len(both_occurance_data))

200
170


In [157]:
print_sents(datas={'manual':True, 'data':both_occurance_data}, num_lines = 10)

development-1445 And it soon does. The name given him by the Department is that of Elizabeth Deveraux; still he inquires further, not believing a woman can have carried the bodies of Janice May Chapman, the woman who was butchered and dumped in an alley; so he discovers she had been framed by the Rileys themselves; Reed Riley - the senator's son- had been together with all the three women killed in town and when he had found out they were pregnant, he had killed them all.

development-1051 The actress was in the cast of the Brazilian version of Desperate Housewives, Donas de Casa Desesperadas in partnership with the Disney with the RedeTV! ; her role was Elisa Fernandes, perfect mother and desperate housewife. In the original version, the role was Bree Van de Kamp, played by Marcia Cross. In television, she had roles in the Brazilian soap operas P*ginas da Vida, Um Anjo Caiu do C*u, Celebridade, Bel*ssima, Am*rica, Amor e *dio, Pequena Travessa and Seus Olhos.

development-123 At the s